# TensorFlow

To import tensorflow, first install it in your system using pip install --upgrade tensorflow


In [1]:
import tensorflow as tf

# create a reset graph function so that we can reset a shell or a pipeline without overwriting the graph

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x=tf.Variable(3, name='x')
y=tf.Variable(5,name='y')
f=x*x*y + y + 5

In [3]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
    


In [5]:
result

55

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [7]:
result

55

# Linear Regression

In [8]:
import numpy as np
from sklearn.datasets import fetch_california_housing


Adding bias to the dataset

In [9]:
housing = fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [10]:
m #rows or instances

20640

In [11]:
n # columns or feaatures

8

In [12]:
housing

{'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/datasets/\n\nThe data contains 20,640 observations on 9 variables.\n\nThis dataset contains the average house value as target variable\nand the following input variables (features): average income,\nhousing average age, average rooms, average bedrooms, population,\naverage occupation, latitude, and longitude in that order.\n\nReferences\n----------\n\nPace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,\nStatistics and Probability Letters, 33 (1997) 291-297.\n\n',
 'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.     

In [13]:
housing_data_plus_bias


array([[   1.        ,    8.3252    ,   41.        , ...,    2.55555556,
          37.88      , -122.23      ],
       [   1.        ,    8.3014    ,   21.        , ...,    2.10984183,
          37.86      , -122.22      ],
       [   1.        ,    7.2574    ,   52.        , ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.        ,    1.7       ,   17.        , ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.        ,    1.8672    ,   18.        , ...,    2.12320917,
          39.43      , -121.32      ],
       [   1.        ,    2.3886    ,   16.        , ...,    2.61698113,
          39.37      , -121.24      ]])

In [14]:
X=tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT=tf.transpose(X)
theta= tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [15]:
with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

standardising the dataset using sklearn's preprocessing module

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_bias_data = np.c_[np.ones((m,1)),scaled_housing_data]

In [17]:
print(scaled_housing_bias_data.mean(axis=0))
print(scaled_housing_bias_data.mean(axis=1))
print(scaled_housing_bias_data.mean())
print(scaled_housing_bias_data.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [18]:
scaled_housing_bias_data.std()

0.9938079899999067

In [19]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_bias_data, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.53737885
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293705


In [20]:
best_theta

array([[ 2.06855226e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44081801e-04],
       [-3.91945131e-02],
       [-8.61356556e-01],
       [-8.23479712e-01]], dtype=float32)

In [21]:
A=tf.placeholder(tf.float32, shape=(None,3))
B = 5+A
with tf.Session() as sees:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
print (B_val_1)

[[6. 7. 8.]]


In [22]:
reset_graph()

X=tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y=tf.placeholder(tf.float32, shape=(None,1), name='y')


theta=tf.Variable(tf.random_uniform([n+1,1],-1,1,seed=23),name='theta')
y_pred=tf.matmul(X,theta,name='Predictor')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [23]:
n_epoch=10
batch_size=100
n_batches=int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch*n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_bias_data[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [24]:
best_theta

array([[ 2.0714476 ],
       [ 0.8462012 ],
       [ 0.11558535],
       [-0.26835832],
       [ 0.32982782],
       [ 0.00608358],
       [ 0.07052915],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)

# Modelling Neurons

In [25]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                     feature_columns=feature_cols)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=40, batch_size=50, shuffle=True)
dnn_clf.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/jt/3p4j5xhd4w12hwqrdkmxzmqr0000gn/T/tmpkppajbug', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10e133470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for 

INFO:tensorflow:global_step/sec: 295.65
INFO:tensorflow:loss = 1.5049099, step = 6001 (0.339 sec)
INFO:tensorflow:global_step/sec: 332.911
INFO:tensorflow:loss = 0.62178326, step = 6101 (0.300 sec)
INFO:tensorflow:global_step/sec: 286.517
INFO:tensorflow:loss = 0.4889561, step = 6201 (0.349 sec)
INFO:tensorflow:global_step/sec: 324.991
INFO:tensorflow:loss = 0.107398644, step = 6301 (0.308 sec)
INFO:tensorflow:global_step/sec: 286.989
INFO:tensorflow:loss = 1.9258989, step = 6401 (0.348 sec)
INFO:tensorflow:global_step/sec: 328.074
INFO:tensorflow:loss = 3.1432679, step = 6501 (0.305 sec)
INFO:tensorflow:global_step/sec: 290.015
INFO:tensorflow:loss = 0.88616806, step = 6601 (0.345 sec)
INFO:tensorflow:global_step/sec: 323.768
INFO:tensorflow:loss = 0.27408206, step = 6701 (0.309 sec)
INFO:tensorflow:global_step/sec: 303.213
INFO:tensorflow:loss = 1.3461821, step = 6801 (0.329 sec)
INFO:tensorflow:global_step/sec: 320.099
INFO:tensorflow:loss = 1.9797683, step = 6901 (0.313 sec)
INFO:t

INFO:tensorflow:global_step/sec: 311.603
INFO:tensorflow:loss = 0.7816235, step = 14201 (0.321 sec)
INFO:tensorflow:global_step/sec: 299.192
INFO:tensorflow:loss = 0.057457197, step = 14301 (0.334 sec)
INFO:tensorflow:global_step/sec: 297.531
INFO:tensorflow:loss = 0.031026768, step = 14401 (0.336 sec)
INFO:tensorflow:global_step/sec: 318.43
INFO:tensorflow:loss = 0.25992426, step = 14501 (0.314 sec)
INFO:tensorflow:global_step/sec: 302.729
INFO:tensorflow:loss = 0.084726736, step = 14601 (0.330 sec)
INFO:tensorflow:global_step/sec: 302.554
INFO:tensorflow:loss = 0.19886562, step = 14701 (0.331 sec)
INFO:tensorflow:global_step/sec: 303.221
INFO:tensorflow:loss = 0.022397151, step = 14801 (0.330 sec)
INFO:tensorflow:global_step/sec: 312.03
INFO:tensorflow:loss = 0.28274137, step = 14901 (0.320 sec)
INFO:tensorflow:global_step/sec: 303.002
INFO:tensorflow:loss = 0.35977662, step = 15001 (0.330 sec)
INFO:tensorflow:global_step/sec: 323.84
INFO:tensorflow:loss = 0.20610406, step = 15101 (0

INFO:tensorflow:global_step/sec: 289.367
INFO:tensorflow:loss = 0.021709338, step = 22301 (0.346 sec)
INFO:tensorflow:global_step/sec: 269.587
INFO:tensorflow:loss = 0.014849992, step = 22401 (0.372 sec)
INFO:tensorflow:global_step/sec: 297.062
INFO:tensorflow:loss = 0.003445588, step = 22501 (0.335 sec)
INFO:tensorflow:global_step/sec: 303.117
INFO:tensorflow:loss = 0.057356678, step = 22601 (0.330 sec)
INFO:tensorflow:global_step/sec: 304.475
INFO:tensorflow:loss = 0.058745325, step = 22701 (0.328 sec)
INFO:tensorflow:global_step/sec: 300.354
INFO:tensorflow:loss = 0.025597462, step = 22801 (0.333 sec)
INFO:tensorflow:global_step/sec: 301.88
INFO:tensorflow:loss = 0.02301766, step = 22901 (0.331 sec)
INFO:tensorflow:global_step/sec: 276.295
INFO:tensorflow:loss = 0.06345755, step = 23001 (0.362 sec)
INFO:tensorflow:global_step/sec: 269.307
INFO:tensorflow:loss = 0.043738663, step = 23101 (0.372 sec)
INFO:tensorflow:global_step/sec: 285.041
INFO:tensorflow:loss = 0.16207972, step = 23

INFO:tensorflow:global_step/sec: 239.481
INFO:tensorflow:loss = 0.055058263, step = 30401 (0.423 sec)
INFO:tensorflow:global_step/sec: 248.159
INFO:tensorflow:loss = 0.029440885, step = 30501 (0.398 sec)
INFO:tensorflow:global_step/sec: 247.828
INFO:tensorflow:loss = 0.010133158, step = 30601 (0.404 sec)
INFO:tensorflow:global_step/sec: 285.965
INFO:tensorflow:loss = 0.009574029, step = 30701 (0.349 sec)
INFO:tensorflow:global_step/sec: 263.421
INFO:tensorflow:loss = 0.019106878, step = 30801 (0.380 sec)
INFO:tensorflow:global_step/sec: 269.967
INFO:tensorflow:loss = 0.01943821, step = 30901 (0.371 sec)
INFO:tensorflow:global_step/sec: 312.141
INFO:tensorflow:loss = 0.0014627706, step = 31001 (0.319 sec)
INFO:tensorflow:global_step/sec: 303.307
INFO:tensorflow:loss = 0.027317762, step = 31101 (0.330 sec)
INFO:tensorflow:global_step/sec: 301.83
INFO:tensorflow:loss = 0.005680061, step = 31201 (0.332 sec)
INFO:tensorflow:global_step/sec: 298.954
INFO:tensorflow:loss = 0.02417224, step = 

INFO:tensorflow:loss = 0.0023110462, step = 38401 (0.327 sec)
INFO:tensorflow:global_step/sec: 265.451
INFO:tensorflow:loss = 0.012830508, step = 38501 (0.376 sec)
INFO:tensorflow:global_step/sec: 286.687
INFO:tensorflow:loss = 0.1109836, step = 38601 (0.349 sec)
INFO:tensorflow:global_step/sec: 267.565
INFO:tensorflow:loss = 0.014529049, step = 38701 (0.374 sec)
INFO:tensorflow:global_step/sec: 260.924
INFO:tensorflow:loss = 0.047503367, step = 38801 (0.383 sec)
INFO:tensorflow:global_step/sec: 312.36
INFO:tensorflow:loss = 0.026197832, step = 38901 (0.321 sec)
INFO:tensorflow:global_step/sec: 242.034
INFO:tensorflow:loss = 0.007399183, step = 39001 (0.412 sec)
INFO:tensorflow:global_step/sec: 284.722
INFO:tensorflow:loss = 0.02218815, step = 39101 (0.352 sec)
INFO:tensorflow:global_step/sec: 291.081
INFO:tensorflow:loss = 0.005090112, step = 39201 (0.344 sec)
INFO:tensorflow:global_step/sec: 261.305
INFO:tensorflow:loss = 0.010496501, step = 39301 (0.383 sec)
INFO:tensorflow:global_s

In [26]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-30-14:38:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/jt/3p4j5xhd4w12hwqrdkmxzmqr0000gn/T/tmpkppajbug/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-30-14:38:22
INFO:tensorflow:Saving dict for global step 44000: accuracy = 0.98, average_loss = 0.09541832, global_step = 44000, loss = 12.078269
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 44000: /var/folders/jt/3p4j5xhd4w12hwqrdkmxzmqr0000gn/T/tmpkppajbug/model.ckpt-44000


In [27]:
eval_results

{'accuracy': 0.98,
 'average_loss': 0.09541832,
 'global_step': 44000,
 'loss': 12.078269}

In [28]:
reset_graph()

import tensorflow as tf

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X=tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y=tf.placeholder(tf.int32, shape=(None), name='y')

In [29]:
def neurons(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs=int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [30]:
with tf.name_scope('deepnn'):
    hidden_layer1 = neurons(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden_layer2 = neurons(hidden_layer1, n_hidden2, name="hidden1",
                           activation=tf.nn.relu)
    
    logits = neurons(hidden_layer2, n_outputs, name="outputs")

In [31]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [32]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [33]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [34]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [35]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [36]:
n_epochs = 20
n_batches = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

0 Batch accuracy: 0.91 Val accuracy: 0.8944
1 Batch accuracy: 0.93 Val accuracy: 0.917
2 Batch accuracy: 0.91 Val accuracy: 0.9278
3 Batch accuracy: 0.93 Val accuracy: 0.9364
4 Batch accuracy: 0.95 Val accuracy: 0.9396
5 Batch accuracy: 0.92 Val accuracy: 0.9434
6 Batch accuracy: 0.97 Val accuracy: 0.9496
7 Batch accuracy: 0.93 Val accuracy: 0.9532
8 Batch accuracy: 0.97 Val accuracy: 0.9554
9 Batch accuracy: 0.95 Val accuracy: 0.956
10 Batch accuracy: 0.95 Val accuracy: 0.9586
11 Batch accuracy: 0.96 Val accuracy: 0.9588
12 Batch accuracy: 0.98 Val accuracy: 0.9592
13 Batch accuracy: 0.95 Val accuracy: 0.9616
14 Batch accuracy: 0.99 Val accuracy: 0.9624
15 Batch accuracy: 0.96 Val accuracy: 0.963
16 Batch accuracy: 0.99 Val accuracy: 0.9624
17 Batch accuracy: 0.97 Val accuracy: 0.9642
18 Batch accuracy: 0.99 Val accuracy: 0.966
19 Batch accuracy: 0.96 Val accuracy: 0.9672
